# Convolutional Neural Network

### Importing the libraries

In [5]:
!pip install --upgrade tensorflow keras Pillow

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [7]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [8]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [9]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [13]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [14]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [15]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [16]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [17]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [18]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [19]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [20]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [21]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

2021-08-27 20:19:23.632324: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25
250/250 [==============================] - 42s 167ms/step - loss: 0.6591 - accuracy: 0.5936 - val_loss: 0.5955 - val_accuracy: 0.6950
Epoch 2/25
250/250 [==============================] - 40s 161ms/step - loss: 0.5820 - accuracy: 0.6948 - val_loss: 0.6193 - val_accuracy: 0.6625
Epoch 3/25
250/250 [==============================] - 42s 169ms/step - loss: 0.5463 - accuracy: 0.7179 - val_loss: 0.5132 - val_accuracy: 0.7540
Epoch 4/25
250/250 [==============================] - 41s 166ms/step - loss: 0.5077 - accuracy: 0.7501 - val_loss: 0.4861 - val_accuracy: 0.7670
Epoch 5/25
250/250 [==============================] - 41s 163ms/step - loss: 0.4869 - accuracy: 0.7673 - val_loss: 0.4825 - val_accuracy: 0.7750
Epoch 6/25
250/250 [==============================] - 41s 165ms/step - loss: 0.4758 - accuracy: 0.7725 - val_loss: 0.5480 - val_accuracy: 0.7285
Epoch 7/25
250/250 [==============================] - 43s 170ms/step - loss: 0.4702 - accuracy: 0.7722 - val_loss: 0.4548 - val_ac

## Part 4 - Making a single prediction

In [22]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [23]:
print(prediction)

dog


## Fim

In [11]:
import os

In [12]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"